In [1]:
from spark import spark

25/04/19 23:04:05 WARN Utils: Your hostname, MacBook-Air-de-Igoh.local resolves to a loopback address: 127.0.0.1; using 172.17.1.180 instead (on interface en0)
25/04/19 23:04:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/anaconda3/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/igoh/.ivy2/cache
The jars for the packages stored in: /Users/igoh/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d566ccca-0b09-4fc1-9cf9-59d2d6df69c1;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.0 in central
	found io.delta#delta-storage;3.3.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 95ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.0 from central in [default]
	io.delta#delta-storage;3.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |

In [2]:
from concurrent.futures import ThreadPoolExecutor
import requests
import pandas as pd
import zipfile
import io
from io import StringIO
import os

In [3]:
dic_tabelas = {
    'tabela_ncm': {
        'nome': 'ncm',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM.csv'
    },
    'tabela_sh': {
        'nome': 'sh',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_SH.csv'
    },
    'tabela_cuci': {
        'nome': 'ncm_cuci',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_CUCI.csv'
    },
    'tabela_isic': {
        'nome': 'ncm_isic',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_ISIC.csv'
    },
    'tabela_isic_cuci': {
        'nome': 'isic_cuci',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/ISIC_CUCI.csv' #apagar
    },
    'tabela_cgce': {
        'nome': 'ncm_cgce',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_CGCE.csv'
    },
    'tabela_fator_agregado': {
        'nome': 'ncm_fat_agreg',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_FAT_AGREG.csv'
    },
    'tabela_ppe': {
        'nome': 'ncm_ppe',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_PPE.csv'
    },
    'tabela_ppi': {
        'nome': 'ncm_ppi',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_PPI.csv'
    },
    'tabela_unidade': {
        'nome': 'ncm_unidade',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_UNIDADE.csv'
    },
    'tabela_nbm_ncm': {
        'nome': 'nbm_ncm',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NBM_NCM.csv'
    },
    'tabela_nbm': {
        'nome': 'nbm',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NBM.csv'
    },
    'tabela_uf': {
        'nome': 'estados',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/UF.csv'
    },
    'tabela_via': {
        'nome': 'via',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/VIA.csv'
    },
    'tabela_urf': {
        'nome': 'urf',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/URF.csv'
    },
    'tabela_pais': {
        'nome': 'paises',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/PAIS.csv'
    },
    'tabela_blocos': {
        'nome': 'blocos',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/PAIS_BLOCO.csv'
    },
    'tabela_municipios': {
        'nome': 'municipios',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/UF_MUN.csv'
    }
}


In [4]:
def import_full_export_import(url, caminho_delta, nome_tabela):
    try:
        # Requisição para obter o arquivo ZIP
        response = requests.get(url)
        response.raise_for_status()

        # Abrir o conteúdo como ZIP
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            print("Arquivos no zip:", z.namelist())

            # Abrir o primeiro arquivo dentro do ZIP
            with z.open(z.namelist()[0]) as csv_file:
                # Caminhos temporários para o DBFS e local
                local_temp_path = "/tmp/temp_file.csv"  # Caminho local na máquina do driver
                dbfs_temp_path = "dbfs:/tmp/temp_file.csv"

                # Escreve temporariamente no driver local
                with open(local_temp_path, 'wb') as f:
                    f.write(csv_file.read())

                # Se for no Databricks, mova o arquivo para o DBFS
                if 'dbutils' in globals():  # Verifica se está no Databricks
                    try:
                        dbutils.fs.cp(f"file:{local_temp_path}", dbfs_temp_path)
                    except Exception as e:
                        print(f"Erro ao mover o arquivo para DBFS: {e}")
                else:
                    print("Ambiente local detectado, mantendo o arquivo local.")

                # Ler com Spark diretamente do DBFS ou local dependendo do ambiente
                path_to_read = dbfs_temp_path if 'dbutils' in globals() else local_temp_path
                df_spark = spark.read.csv(path_to_read, header=True, sep=';', inferSchema=True, encoding='latin1')

                # Salva como tabela Delta
                df_spark.write.format("delta") \
                    .option("overwriteSchema", True) \
                    .mode("overwrite") \
                    .saveAsTable(f"{caminho_delta}.{nome_tabela}")

                print(f"Tabela {nome_tabela} salva com sucesso em: {caminho_delta}.{nome_tabela}")

                # Remove os arquivos temporários
                os.remove(local_temp_path)
                if 'dbutils' in globals():
                    try:
                        dbutils.fs.rm(dbfs_temp_path)
                    except Exception as e:
                        print(f"Erro ao remover o arquivo do DBFS: {e}")

    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar a URL: {e}")
    except zipfile.BadZipFile as e:
        print(f"Erro ao abrir o arquivo ZIP: {e}")
    except Exception as e:
        print(f"Erro ao processar o arquivo CSV: {e}")


In [5]:
def baixar_csv(info):
    url = info['url']
    nome = info['nome']
    
    response = requests.get(url)
    response.raise_for_status()
    
    df = pd.read_csv(StringIO(response.text), sep=';')
    return nome, df

def importar_table_assistants(dic_tabelas, nome_banco):

    with ThreadPoolExecutor() as executor:
        resultados = executor.map(baixar_csv, dic_tabelas.values())

        for nome, df in resultados:
            df_spark = spark.createDataFrame(df)
            tabela = f"{nome_banco}.{nome}"
            df_spark.write.format("delta").option('overwriteSchema', True).mode("overwrite").saveAsTable(tabela)
            print(f"{nome} salva como tabela: {tabela}")

In [6]:

spark.sql("CREATE DATABASE IF NOT EXISTS bd_becomex LOCATION 'storage'")

DataFrame[]

In [7]:
importar_table_assistants(dic_tabelas, "bd_becomex")
import_full_export_import('https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm/IMP_COMPLETA.zip', 'bd_becomex', 'import') # importação
import_full_export_import('https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm/EXP_COMPLETA.zip', 'bd_becomex', 'export') # exportação

25/04/19 23:04:12 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


ncm salva como tabela: bd_becomex.ncm
CodeCache: size=131072Kb used=45593Kb max_used=45897Kb free=85478Kb
 bounds [0x000000010b1f8000, 0x000000010df28000, 0x00000001131f8000]
 total_blobs=16913 nmethods=15766 adapters=1058
 compilation: disabled (not enough contiguous free space left)


Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=
25/04/19 23:04:20 WARN TaskSetManager: Stage 18 contains a task of very large size (1148 KiB). The maximum recommended task size is 1000 KiB.


sh salva como tabela: bd_becomex.sh


ncm_cuci salva como tabela: bd_becomex.ncm_cuci


...

paises salva como tabela: bd_becomex.paises


blocos salva como tabela: bd_becomex.blocos


municipios salva como tabela: bd_becomex.municipios
Arquivos no zip: ['IMP_COMPLETA.csv']
Ambiente local detectado, mantendo o arquivo local.


Tabela import salva com sucesso em: bd_becomex.import
Arquivos no zip: ['EXP_COMPLETA.csv']
Ambiente local detectado, mantendo o arquivo local.


Tabela export salva com sucesso em: bd_becomex.export


In [9]:
spark.sql("""use bd_becomex""")

DataFrame[]

In [11]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [13]:
spark.sql("""

WITH ncm_joined AS (
    SELECT *
    FROM ncm n
    INNER JOIN sh s ON n.CO_SH6 = s.CO_SH6
    INNER JOIN ncm_ppe np ON n.CO_PPE = np.CO_PPE
    INNER JOIN ncm_ppi npi ON n.CO_PPI = npi.CO_PPI
    INNER JOIN ncm_fat_agreg nfa ON n.CO_FAT_AGREG = nfa.CO_FAT_AGREG
    INNER JOIN ncm_cuci nc ON n.CO_CUCI_ITEM = nc.CO_CUCI_ITEM
    INNER JOIN ncm_cgce ncg ON n.CO_CGCE_N3 = ncg.CO_CGCE_N3
    INNER JOIN ncm_isic ni ON n.CO_ISIC_CLASSE = ni.CO_ISIC_CLASSE
),
fat_import_joined AS (
    SELECT *
    FROM bd_becomex.import fi
    INNER JOIN ncm_unidade nu ON fi.CO_UNID = nu.CO_UNID
    INNER JOIN urf u ON fi.CO_URF = u.CO_URF
    INNER JOIN paises p ON fi.CO_PAIS = p.CO_PAIS
    INNER JOIN via v ON fi.CO_VIA = v.CO_VIA
),
fat_export_joined AS (
    SELECT *
    FROM bd_becomex.export fe
    INNER JOIN ncm_unidade nu ON fe.CO_UNID = nu.CO_UNID
    INNER JOIN urf u ON fe.CO_URF = u.CO_URF
    INNER JOIN paises p ON fe.CO_PAIS = p.CO_PAIS
    INNER JOIN via v ON fe.CO_VIA = v.CO_VIA
)
SELECT *
FROM fat_import_joined fi
INNER JOIN ncm_joined nj ON fi.CO_NCM = nj.CO_NCM
WHERE fi.CO_ANO = '2025'

""").limit(5).toPandas()


,CO_ANO,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB,VL_FRETE,VL_SEGURO,CO_UNID,NO_UNID,SG_UNID,CO_URF,NO_URF,CO_PAIS,CO_PAIS_ISON3,CO_PAIS_ISOA3,NO_PAIS,NO_PAIS_ING,NO_PAIS_ESP,CO_VIA,NO_VIA,CO_NCM,CO_UNID,CO_SH6,CO_PPE,CO_PPI,CO_FAT_AGREG,CO_CUCI_ITEM,CO_CGCE_N3,CO_SIIT,CO_ISIC_CLASSE,CO_EXP_SUBSET,NO_NCM_POR,NO_NCM_ESP,NO_NCM_ING,CO_SH6,NO_SH6_POR,NO_SH6_ESP,NO_SH6_ING,CO_SH4,NO_SH4_POR,NO_SH4_ESP,NO_SH4_ING,CO_SH2,NO_SH2_POR,NO_SH2_ESP,NO_SH2_ING,CO_NCM_SECROM,NO_SEC_POR,NO_SEC_ESP,NO_SEC_ING,CO_PPE,NO_PPE,NO_PPE_MIN,NO_PPE_ING,CO_PPI,NO_PPI,NO_PPI_MIN,NO_PPI_ING,CO_FAT_AGREG,NO_FAT_AGREG,NO_FAT_AGREG_GP,CO_CUCI_ITEM,NO_CUCI_ITEM,CO_CUCI_SUB,NO_CUCI_SUB,CO_CUCI_GRUPO,NO_CUCI_GRUPO,CO_CUCI_DIVISAO,NO_CUCI_DIVISAO,CO_CUCI_SEC,NO_CUCI_SEC,CO_CGCE_N3,NO_CGCE_N3,NO_CGCE_N3_ING,NO_CGCE_N3_ESP,CO_CGCE_N2,NO_CGCE_N2,NO_CGCE_N2_ING,NO_CGCE_N2_ESP,CO_CGCE_N1,NO_CGCE_N1,NO_CGCE_N1_ING,NO_CGCE_N1_ESP,CO_ISIC_CLASSE,NO_ISIC_CLASSE,NO_ISIC_CLASSE_ING,NO_ISIC_CLASSE_ESP,CO_ISIC_GRUPO,NO_ISIC_GRUPO,NO_ISIC_GRUPO_ING,NO_ISIC_GRUPO_ESP,CO_ISIC_DIVISAO,NO_ISIC_DIVISAO,NO_ISIC_DIVISAO_ING,NO_ISIC_DIVISAO_ESP,CO_ISIC_SECAO,NO_ISIC_SECAO,NO_ISIC_SECAO_ING,NO_ISIC_SECAO_ESP
0,2025,1,2032900,10,158,SC,7,1017503,46388,46388,82576,4843,45,10,QUILOGRAMA LIQUIDO,KGL,1017503,1017503 - IRF - SÃO BORJA,158,152,CHL,Chile,Chile,Chile,7,RODOVIARIA,2032900,10,20329,1117.0,1990.0,1.0,01222,220,4000.0,1010,507.0,"Outras carnes de suíno, congeladas","Las demás carnes de la esp.porcina, congeladas","Other swine meat, frozen",20329,"Outras carnes de suíno, congeladas","Carne de porcinos, congelada (exc. canales o m...",Frozen meat of swine (excl. carcases and half-...,203,"Carnes de animais da espécie suína, frescas, r...","Carne de animales de la especie porcina, fresc...","Meat of swine, fresh, chilled or frozen",2,"Carnes e miudezas, comestíveis",Carne y despojos comestibles,Meat and edible meat offal,I,Animais vivos e produtos do reino animal,Animales vivos y productos del reino animal,Lives animals; Animal products,1117,"CARNE DE SUINO CONGELADA, FRESCA OU REFRIGERADA","Carne de suíno congelada, fresca ou refrigerada","PIGMEAT FROZEN, FRESH OR CHILLED",1990,DEMAIS PRODUTOS BASICOS,Demais produtos básicos,OTHER BASIC GOODS,1,PRODUTOS BASICOS,PRODUTOS BASICOS,01222,Carne de suínos congelada,122,"Carne suína fresca, resfriada ou congelada",012c2,"Carne suína fresca, refrigerada ou congelada",1,Carne e preparações de carne,0,PRODUTOS ALIMENTICIOS E ANIMAIS VIVOS,220,"Alimentos e bebidas elaborados, destinados pri...","Elaborated food and beverages, mainly for indu...","Alimentos y bebidas preparados, principalmente...",22,"Alimentos e bebidas elaborados, destinados pri...","Elaborated food and beverages, mainly for indu...","Alimentos y bebidas preparados, principalmente...",2,BENS INTERMEDIÁRIOS (BI),INTERMEDIATE GOODS (BI),BIENES INTERMEDIOS (BI),1010,Processamento e conservação de carne,Processing and preserving of meat,Processamento e conservação de carne,101,Processamento e conservação de carne,Processing and preserving of meat,Processamento e conservação de carne,10,Fabricação de produtos alimentícios,Manufacture of food products,Fabricação de produtos alimentícios,C,Indústria de Transformação,Manufacturing,Indústria de Transformação
1,2025,2,3038112,10,161,SP,1,817800,848476,848476,1032203,165354,164,10,QUILOGRAMA LIQUIDO,KGL,817800,0817800 - PORTO DE SANTOS,161,158,TWN,Taiwan (Formosa),Taiwan,Taiwán,1,MARITIMA,3038112,10,30381,1380.0,1990.0,1.0,03428,324,4000.0,1020,2403.0,"Tubarão-azul, eviscerado, sem cabeça e sem bar...","Tibuones azules destripados,sin cola,sin cabez...","Blue shark gutted, finess, headless, frozen",30381,Cação e outros tubarões,Escualos,Dogfish and other sharks,303,"Peixes congelados, exceto os filés de peixes e...",Pescado congelado (excepto los filetes y demás...,"Fish, frozen, excluding fish fillets and other...",3,"Peixes e crustáceos, moluscos e outros inverte...","Pescados y crustáceos, mol

In [ ]:
#ncm_joined = (
#    ncm
#    .join(sh, on='CO_SH6', how='inner')
#    .join(ncm_ppe, on='CO_PPE', how='inner')
#    .join(ncm_ppi, on='CO_PPI', how='inner')
#    .join(ncm_fat_agreg, on='CO_FAT_AGREG', how='inner')
#    .join(ncm_cuci, on='CO_CUCI_ITEM', how='inner')
#    .join(ncm_cgce, on='CO_CGCE_N3', how='inner')
#    .join(ncm_isic, on='CO_ISIC_CLASSE', how='inner')
#)
#
#fat_import = spark.read.format("delta").table("bd_becomex.import")
#
#fat_import_joined = (
#    fat_import
#    .join(ncm_unidade, on='CO_UNID', how='inner')
#    .join(urf, on='CO_URF', how='inner')
#    .join(paises, on='CO_PAIS', how='inner')
#    .join(via, on='CO_VIA', how='inner')
#)
#
#fat_export = spark.read.format("delta").table("bd_becomex.export")
#
#fat_export_joined = (
#    fat_export
#    .join(ncm_unidade, on='CO_UNID', how='inner')
#    .join(urf, on='CO_URF', how='inner')
#    .join(paises, on='CO_PAIS', how='inner')
#    .join(via, on='CO_VIA', how='inner')
#)
#
#total_import = (
#    fat_import_joined
#    .merge(ncm_joined, left_on='CO_NCM', right_on='CO_NCM', how='inner')
#)
#total_import = total_import.drop("CO_UNID")

In [ ]:
#fat_import = spark.read.format("delta").table("bd_becomex.import")
#
#fat_import_joined = (
#    fat_import
#    .join(ncm_unidade, on='CO_UNID', how='inner')
#    .join(urf, on='CO_URF', how='inner')
#    .join(paises, on='CO_PAIS', how='inner')
#    .join(via, on='CO_VIA', how='inner')
#)
#

In [ ]:
#fat_export = spark.read.format("delta").table("bd_becomex.export")
#
#fat_export_joined = (
#    fat_export
#    .join(ncm_unidade, on='CO_UNID', how='inner')
#    .join(urf, on='CO_URF', how='inner')
#    .join(paises, on='CO_PAIS', how='inner')
#    .join(via, on='CO_VIA', how='inner')
#)


In [ ]:
#total_import = (
#    fat_import_joined
#    .merge(ncm_joined, left_on='CO_NCM', right_on='CO_NCM', how='inner')
#)
#total_import = total_import.drop("CO_UNID")

In [ ]:
#total_export = (
#    fat_export_joined
#    .merge(ncm_joined, left_on='CO_NCM', right_on='CO_NCM', how='inner')
#)
#total_export = total_export.drop("CO_UNID")

In [ ]:
#%sql
#OPTIMIZE bd_becomex.Baseimport;
#OPTIMIZE bd_becomex.Baseexport;